In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
import holoviews as hv
from holoviews import opts


In [ ]:
# Загрузка данных
Labels = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']
(X_train, y_train), (X_test_class, y_test_class) = fashion_mnist.load_data()
(X_train, y_train), (X_test_class, y_test_class) = fashion_mnist.load_data()
X_train = np.pad(X_train, ((0, 0), (2, 2), (2, 2)), mode='constant')  # Увеличиваем размер
X_train = np.repeat(X_train[..., np.newaxis], 3, axis=-1) / 255.0  # Нормализация и преобразование в RGB
X_test = np.pad(X_test_class, ((0, 0), (2, 2), (2, 2)), mode='constant')  # Увеличиваем размер
X_test = np.repeat(X_test[..., np.newaxis], 3, axis=-1) / 255.0  # Нормализация и преобразование в RGB

# Преобразование меток в категориальный формат
y_train_categorical = to_categorical(y_train, num_classes=10)
y_test_categorical = to_categorical(y_test_class, num_classes=10)

In [ ]:
print(X_train.min(), X_train.max())
print(X_test.min(), X_test.max())

In [ ]:
datagen = ImageDataGenerator(
    rotation_range=20,  # Увеличьте угол поворота
    width_shift_range=0.2,  # Увеличьте сдвиг по ширине
    height_shift_range=0.2,  # Увеличьте сдвиг по высоте
    horizontal_flip=True,
    zoom_range=0.2,  # Добавьте зум
    shear_range=0.2  # Добавьте сдвиг
)

X_train = np.resize(X_train, (X_train.shape[0], 32, 32, 3))  # Пример изменения размера
X_test = np.resize(X_test, (X_test.shape[0], 32, 32, 3))  # Пример изменения размера

datagen.fit(X_train)

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras import layers, models
import holoviews as hv
from holoviews import opts


In [ ]:
from tensorflow.keras.applications import VGG16
def create_model():
    base_model = VGG16(weights='imagenet', include_top=False, input_shape=(32, 32, 3))
    model = Sequential()
    model.add(base_model)
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(10, activation='softmax'))
    
    return model



In [ ]:
model = create_model()
model.summary()
model.compile(optimizer="adam",
              loss="categorical_crossentropy",  # Используем категориальную кросс-энтропию
              metrics=["accuracy"])



In [ ]:
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('best_model.h5', save_best_only=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6)

history = model.fit(datagen.flow(X_train, y_train_categorical, batch_size=64),
                    validation_data=(X_test, y_test_categorical),
                    epochs=100,
                    callbacks=[early_stop, model_checkpoint, reduce_lr])

In [ ]:
train_score = model.evaluate(X_train, y_train_categorical)
print('Train score:', train_score)
test_score = model.evaluate(X_test, y_test_categorical)
print('Test score:', test_score)
pred_probs = model.predict(X_test)
y_pred = np.argmax(pred_probs, axis=1) # Получаем классы с максимальной вероятностью


In [ ]:
target_class = 8 # класс 'Bag'
errors = np.where((y_test_class == target_class) & (y_pred != target_class))[0]
misclassified_classes = y_pred[errors] # Классы, к которым были ошибочно отнесены
misclassified_counts = np.bincount(misclassified_classes, minlength=10) # Считаем количество ошибок по классам
for i, count in enumerate(misclassified_counts):
    if count > 0:
        print(f'Класс {Labels[i]} ошибочно классифицирован {count} раз(а).')

In [ ]:
hv.extension('bokeh')

In [ ]:
# Функция для отображения изображений с ошибками
def draw_image_test(i):
    return hv.Image(X_test[i]).opts(title="Predicted: %s | Actual: %s" % (Labels[y_pred[i]], Labels[y_test_class[i]]),
                                      xaxis='bare', yaxis='bare',
                                      cmap='grey', height=300, width=300,
                                      cnorm='linear')

# Отображение изображений с ошибками
img = {i: draw_image_test(i) for i in errors}
NdLayout = hv.NdLayout(img).opts(title="Images predicted class", height=2250, width=1200).cols(4)
NdLayout

In [ ]:
# Сохранение модели
model.save('fashion_mnist_model_30.keras')